# Day 4

I'm going to start in by reading in our data and storing the rolls and the boards as lists

In [729]:
import copy

In [730]:
file = open('dayFourData.txt', 'r')
fileText = file.read().split('\n\n')
rolls = [int(i) for i in fileText.pop(0).split(',')]
boards = []
for board in fileText:
    board = board.split('\n')
    for i in range (len(board)):
        board[i] = [int(value) for value in board[i].split()]
    boards.append(board)


## Part One
You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:
```
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
 ```

After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
 ```
After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
 ```
Finally, 24 is drawn:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
 ```
At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: 14 21 17 24 4).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, **figure out which board will win first. What will your final score be if you choose that board?**

Step 1: Making an function that will mark off the called row on each board as a **-1**

In [731]:
def checkRoll(roll, board):
  for rowIndex in range(len(board)):
    for i in range(len(board[rowIndex])):
      if board[rowIndex][i] == roll:
        board[rowIndex][i] = -1

Step 2: Making a function that will check the board's rows and columns for a bingo (entire row or column is comprised of -1)

In [732]:
def bingoChecker(board):
  #check all rows for bingo
  for row in board:
    if sum(row) == len(row) * -1:
        return True
  #check all columns for bingo
  for i in range(len(board)):
      columnSum = 0
      for row in board:
        columnSum += row[i]
      if columnSum == len(board) * -1:
        return True
  return False

Step 3: Getting the unmarked sum of a board (summing all the values that are not -1)

In [733]:
def unmarkedSum(board):
  sum = 0
  for row in board:
    for value in row:
      if value != -1:
        sum += value
  return sum

Step 4: Making the bingo game function (will return the final score of the winning board)

In [734]:
def bingoGame(rolls, boards):
  for roll in rolls:
    for board in boards: 
      checkRoll(roll, board)
      if bingoChecker(board):
        return roll * unmarkedSum(board)

Final result:

In [735]:
finalScore = bingoGame(rolls, copy.deepcopy(boards))
print(f'The final score of a winning board is: {finalScore}')

The final score of a winning board is: 8136


## Part Two

On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.

**Figure out which board will win last. Once it wins, what would its final score be?**

I'm going to modify the original `bingoGame` function to make it so that it keeps track of the boards that have gotten a bingo and then keeps on playing the game with the boards that haven't yet until the very last board wins

In [736]:
def bingoGamePartTwo(rolls, boards):
  winners = []
  for roll in rolls:
    boards = [i for i in boards if i not in winners]
    for board in boards: 
      checkRoll(roll, board)
      if bingoChecker(board):
        if (len(boards) == 1):
          return roll * unmarkedSum(board) 
        winners.append(board)

In [737]:
partTwoFinalScore = bingoGamePartTwo(rolls, copy.deepcopy(boards))

print(f'The final score for the last board to win is: {partTwoFinalScore}')

The final score for the last board to win is: 12738
